#Kaggle

In [1]:
from google.colab import files

In [2]:
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"lmfromcapo","key":"76d424ecb7028ac9dee592afff7bb569"}'}

In [3]:
import os
import shutil

os.makedirs('/root/.kaggle/', exist_ok=True)

shutil.move('kaggle.json', '/root/.kaggle/kaggle.json')

os.chmod('/root/.kaggle/kaggle.json', 600)

In [4]:
!pip install kaggle

In [5]:
!kaggle competitions download -c titanic

  0% 0.00/34.1k [00:00<?, ?B/s]
100% 34.1k/34.1k [00:00<00:00, 36.5MB/s]


In [6]:
!unzip titanic.zip

Archive:  titanic.zip
  inflating: gender_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


#Titanic

In [7]:
import pandas as pd
import numpy as np

In [8]:
gender = pd.read_csv('/content/gender_submission.csv')
test = pd.read_csv("/content/test.csv")
train = pd.read_csv("/content/train.csv")

In [9]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [10]:
train = train.drop(['PassengerId', 'Ticket'], axis = 1)

test = test.drop(['Ticket'], axis = 1)

In [11]:
train['Sex'] = train['Sex'].map({'male':0, 'female':1})

test['Sex'] = test['Sex'].map({'male':0, 'female':1})


In [12]:
train.isna().sum()
train['Age'] = train['Age'].fillna(train['Age'].median())
train['Cabin'] = train['Cabin'].fillna('Unknown')
train['Embarked'] = train['Embarked'].fillna(train['Embarked'].mode()[0])


test['Age'] = train['Age'].fillna(train['Age'].median())
test['Cabin'] = train['Cabin'].fillna('Unknown')
test['Fare'] = train['Fare'].fillna(train['Fare'].mean())


In [13]:
train['Name'] = train['Name'].apply(lambda x: x.split(',')[1].split('.')[0].strip())

In [14]:
train['FamilySize'] = train['SibSp'] + train['Parch'] + 1

test['FamilySize'] = test['SibSp'] + test['Parch'] + 1

In [15]:
train['IsLonely'] = 0
train.loc[train['FamilySize'] == 1, 'IsLonely'] = 1

test['IsLonely'] = 0
test.loc[test['FamilySize'] == 1, 'IsLonely'] = 1

In [16]:
train['Age'] = np.log1p(train['Age'])
train['SibSp'] = np.log1p(train['SibSp'])
train['Parch'] = np.log1p(train['Parch'])
train['Fare'] = np.log1p(train['Fare'])
train['FamilySize'] = np.log1p(train['FamilySize'])

test['Age'] = np.log1p(test['Age'])
test['SibSp'] = np.log1p(test['SibSp'])
test['Parch'] = np.log1p(test['Parch'])
test['Fare'] = np.log1p(test['Fare'])
test['FamilySize'] = np.log1p(test['FamilySize'])

In [17]:
train['Name'] = train['Name'].replace(['Lady', 'the Countess','Capt', 'Col', 'Don', 'Dr',
                                   'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
train['Name'] = train['Name'].replace('Mlle', 'Miss')
train['Name'] = train['Name'].replace('Ms', 'Miss')
train['Name'] = train['Name'].replace('Mme', 'Mrs')

#------------------------------

#train['Name'].replace(['Capt', 'Col', 'Major', 'Dr', 'Rev'], 'Officer', inplace=True)
#train['Name'].replace(['Don', 'Sir', 'the Countess', 'Jonkheer', 'Lady'], 'Royalty', inplace=True)
#train['Name'].replace(['Mme', 'Ms', 'Mrs'], 'Mrs', inplace=True)
#train['Name'].replace(['Mlle', 'Miss'], 'Miss', inplace=True)

In [18]:
train['Name'].value_counts()

,count
Name,
Mr,517
Miss,185
Mrs,126
Master,40
Rare,23


In [19]:
train['Cabin'] = train['Cabin'].apply(lambda x: x[0])

In [20]:
train = pd.get_dummies(train, columns=['Pclass'], drop_first=True)
#train = pd.get_dummies(train, columns=['Name'],drop_first=True)
train = pd.get_dummies(train, columns=['Sex'], drop_first=True)
#train = pd.get_dummies(train, columns=['Cabin'], drop_first=True)
#train = pd.get_dummies(train, columns=['Embarked'], drop_first=True)
train = pd.get_dummies(train, columns=['IsLonely'], drop_first=True)


test = pd.get_dummies(test, columns=['Pclass'], drop_first=True)
test = pd.get_dummies(test, columns=['Sex'], drop_first=True)
test = pd.get_dummies(test, columns=['IsLonely'], drop_first=True)

#Inferencia

In [21]:
!pip install ISLP

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.3/349.3 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 522.0/522.0 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.4/36.4 MB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 869.5/869.5 kB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.5/94.5 kB 6.9 MB/s eta 0:00:00
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4031 sha256=3dfab97848342a35903f7432041ba59c4ab9d0c92d1667f3a6fa281417569eed
  Stored in directory: /root/.cache/pip/wheels/25/cc/e0/ef2969164144c899fedb22b338f6703e2b9cf46eeebf254991
Successfully built autograd-gamma
  Attempting un

In [22]:
from matplotlib.pyplot import subplots
import statsmodels.api as sm
from ISLP import load_data
from ISLP.models import (ModelSpec as MS, summarize)
from ISLP import confusion_table
from ISLP.models import contrast
from sklearn.discriminant_analysis import (LinearDiscriminantAnalysis as LDA,
                                           QuadraticDiscriminantAnalysis as QDA)
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression


In [23]:
train.columns

Index(['Survived', 'Name', 'Age', 'SibSp', 'Parch', 'Fare', 'Cabin',
       'Embarked', 'FamilySize', 'Pclass_2', 'Pclass_3', 'Sex_1',
       'IsLonely_1'],
      dtype='object')

In [24]:
design = MS(train.drop(['Survived','Name','SibSp','Parch', 'Cabin','Embarked'], axis=1))
X = design.fit_transform(train)
#X = sm.add_constant(X)
y = train['Survived']

glm = sm.GLM(y,
             X,
             family=sm.families.Binomial())

In [25]:
results = glm.fit()
summarize(results)

,coef,std err,z,P>|z|
intercept,4.7630,1.125,4.235,0.000
Age,-1.0043,0.169,-5.928,0.000
Fare,0.3397,0.169,2.013,0.044
FamilySize,-2.4243,0.501,-4.843,0.000
Pclass_2,-0.7600,0.318,-2.393,0.017
Pclass_3,-1.6892,0.346,-4.876,0.000
Sex_1,2.7606,0.204,13.549,0.000
IsLonely_1,-1.0920,0.361,-3.021,0.003


In [26]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [27]:
vif_data = pd.DataFrame()
vif_data['feature'] = X.columns
vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

In [28]:
vif_data

,feature,VIF
0,intercept,133.619176
1,Age,1.249304
2,Fare,2.997206
3,FamilySize,4.219809
4,Pclass_2,2.271047
5,Pclass_3,3.648083
6,Sex_1,1.130542
7,IsLonely_1,3.756901


In [29]:
probs = results.predict(X)

In [30]:
labels = np.array(['no ']*len(probs))
labels[probs > 0.5] = 'yes'

In [31]:
dead_live = y.replace({0:'no ', 1:'yes'})

In [32]:
confusion_table(labels,dead_live)

Truth,no,yes
Predicted,,
no,481,99
yes,68,243


In [33]:
print(results.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:               Survived   No. Observations:                  891
Model:                            GLM   Df Residuals:                      883
Model Family:                Binomial   Df Model:                            7
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -384.91
Date:                Fri, 11 Oct 2024   Deviance:                       769.83
Time:                        20:38:16   Pearson chi2:                     940.
No. Iterations:                     5   Pseudo R-squ. (CS):             0.3736
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      4.7630      1.125      4.235      0.0

**Todas as variaveis presentes no modelo foram escolhidas por apresentarem um p>|z| < 0.05 rejeitando a Hipotese nula ao nivel de significancia de 5%, as demais variaveis disponibilizadas no dataset não demonstrarão significancia estatistica para explicar o modelo**
**************
1.   **Cabeçalho**

*      **Dep. Variable** indica que a variavel dependente é **Survived**, ou seja, o modelo tentará prever quem sobreviveu (1) e quem não sobreviveu (0) na tragédia do Titanic.

*     **No. Observations** total de observações usados para treinar o modelo foi de **891**

*     **Model** o modelo utilizado foi o GLM (Modelo Linear Generalizado)

*     **Model family** O modelo pertence à familia **Binomial**, comum em modelos de classificação Binaria

*   **Link Function** função logt ou regressão logistica ordinal mapeia probabilidades para a escala dos numeros reais, o que é adequado para um problema Binomial

*  **Log-Likelihood (log verossimilhança)** é um indicador de qual bem o modelo se ajusta aos dados e quanto maior (menos negativo melhor), nesse caso temos um valor negativo mas não extremo de **-384.91** o que indica a necessidade do uso de outros modelos para comparar a eficacia dele.

*   **Deviance** Medida da falta de  ajuste do modelo calculado pelo dobro da diferença entre Log-Likelihood e o modelo saturado (ideal). Um valor menor de Deviance sugere melhor ajuste, no caso temos **769.83** não muito alta mas é necessario a comparação com um modelo nulo

*   **Pearson Chi²** verifica a adequação do ajuste do modelo. Ele compara os valores previstos com os observados, ou seja, um Chi² de 940 indica que o modelo tem dificuldades em se ajustar aos dados em algumas partes corretamente

*   **Pseudo R_square** é uma metrica que tenta capturar a proporção de variabilidade dos dados que explicado pelo modelo, nesse caso um Pseudo R-square de 0.3736 indica que o modelo explica somente 37% da variação dos dados aproxiamdamente deixando outros **63%** sem explicação.
****
2.   **Tabela de coeficientes**

*   **Idade** **Tamanho da familia** e **Se está sozinho** com coeficientes negativos estão associados na diminuição nas chances de sobriviver, ou seja, quanto maior a idade, tamanho da familia e se está sozinho diminuem as probabilidades de sobrevivencia por um fator de exp(-1.0043) ≈ 0.366 e exp(-2.4243) ≈ 0.0886 respectivamente

*  **Pclass_2** e **Pclass_3** com coeficientes negativos indicam que os passageiros da 2º e 3º tinha menores probabilidades de vitoria em relação aos passageiros da 1º classe por um fator de exp(-0.7600) ≈ 0.4679 e exp(-1.6892) ≈ 0.1844 respectivamente.

*  **Fare** e **sex_1** com coefecientes positivos indicam que pagar mais no valor da passagem (Fare) e ser do sexo feminino (1) aumentam a probabilidade de sobrevivencia por um fator de exp(0.3397) ≈ 1.405 e exp(2.7606) ≈ 15.8113 respectivamente

****
3.   **Conclusão**

*  Variáveis como Age, FamilySize, Pclass_2, Pclass_3, e IsLonely estão associadas a uma redução na probabilidade de sobrevivência, como mostrado pelos coeficientes negativos e razões de chances menores que 1.

Fare e Sex (ser mulher) aumentam as chances de sobrevivência, como indicado pelos coeficientes positivos e razões de chances maiores que 1.

A magnitude dos coeficientes e a significância estatística variam, com algumas variáveis, como Sex_1 e Pclass_3, tendo um impacto mais forte no modelo, enquanto outras, como Fare, são menos expressivas, mas ainda significativas.
****



In [34]:
from sklearn.metrics import accuracy_score

accuracy_score(dead_live,labels)

0.8125701459034792

In [35]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [36]:
print(classification_report(dead_live,labels))

              precision    recall  f1-score   support

         no        0.83      0.88      0.85       549
         yes       0.78      0.71      0.74       342

    accuracy                           0.81       891
   macro avg       0.81      0.79      0.80       891
weighted avg       0.81      0.81      0.81       891



*  **Precision (Precisão)**: Refere-se à proporção de verdadeiros positivos (predições corretas da classe) entre todas as instâncias que foram classificadas como positivas pelo modelo. Ou seja, quantas das previsões positivas estão corretas.

*  Para a classe **no**, a precisão é de **0.83** (83%), ou seja, das previsões feitas para **no**, 83% estavam corretas.

*  Para a classe **yes**, a precisão é de **0.78** (78%), ou seja, 78% das previsões de "yes" estavam corretas.

*  **Recall (Revocação ou Sensibilidade)**: Mede a capacidade do modelo de encontrar todas as instâncias positivas verdadeiras. Ou seja, quantos dos exemplos positivos foram identificados corretamente.

Para a classe **no**, o recall é de **0.88** (88%), indicando que 88% dos exemplos verdadeiros de **no** foram corretamente identificados.
Para a classe **yes**, o recall é de **0.71** (71%), o que significa que 71% dos exemplos verdadeiros de "yes" foram corretamente identificados.

*  **F1-score**: É a média harmônica da precisão e do recall. Dá uma medida mais balanceada da performance do modelo, especialmente útil quando há um desequilíbrio entre as classes.

Para a classe **no**, o **F1-score** é de **0.85**, o que sugere um bom equilíbrio entre precisão e recall.
Para a classe **yes**, o **F1-score** é de **0.74**, um pouco menor, indicando que o modelo tem mais dificuldade em identificar corretamente a classe "yes".

*  **Support**: Refere-se ao número de ocorrências reais de cada classe no conjunto de dados.

A classe **no** tem 549 ocorrências.
A classe **yes** tem 342 ocorrências.

2. **Linha "accuracy"**
Accuracy (Acurácia): A acurácia geral do modelo é 0.81 (81%), indicando que o modelo classificou corretamente 81% das instâncias no conjunto de dados total de 891 amostras.

3. Linhas "macro avg" e "weighted avg"
*  **Macro avg**: A média simples das métricas (precision, recall, F1-score) para as duas classes, sem levar em consideração o desbalanceamento das classes. Isso dá a mesma importância a cada classe, independentemente do número de ocorrências.

Para este caso, o F1-score médio é 0.80.

* **Weighted avg**: A média ponderada das métricas para as duas classes, levando em conta o número de ocorrências (suporte) de cada classe. Classes com mais exemplos têm mais influência no cálculo.

Aqui, o **F1-score** ponderado é **0.81**, refletindo o impacto das classes com mais exemplos.

4. Conclusão
O modelo apresenta boa performance com uma acurácia de 81%.
Ele tem um desempenho melhor para a classe "no", com maiores valores de precisão, recall e F1-score, indicando que consegue classificar melhor essa classe em comparação com a classe **yes**
.
A classe **yes** tem um recall mais baixo (71%), o que significa que o modelo não está identificando corretamente uma parte significativa dos exemplos verdadeiros dessa classe.

#sklearn

In [37]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler

In [38]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X)


In [39]:
logistic_model = LogisticRegression(random_state = 42)

logistic_model.fit(X_train, y)

LogisticRegression(random_state=42)

In [40]:
y_predict = logistic_model.predict(X_train)

In [41]:
print(confusion_matrix(y,y_predict))

[[481  68]
 [ 99 243]]


In [42]:
print(accuracy_score(y,y_predict))

0.8125701459034792


#Testando os modelos

##statsmodel

In [43]:
test.columns

Index(['PassengerId', 'Name', 'Age', 'SibSp', 'Parch', 'Fare', 'Cabin',
       'Embarked', 'FamilySize', 'Pclass_2', 'Pclass_3', 'Sex_1',
       'IsLonely_1'],
      dtype='object')

In [44]:
design_test = MS(test.drop(['PassengerId','Name','SibSp','Parch', 'Cabin','Embarked'], axis=1))
X_test = design_test.fit_transform(test)
y_test = gender['Survived']

In [45]:
X.columns

Index(['intercept', 'Age', 'Fare', 'FamilySize', 'Pclass_2', 'Pclass_3',
       'Sex_1', 'IsLonely_1'],
      dtype='object')

In [46]:
X_test.columns

Index(['intercept', 'Age', 'Fare', 'FamilySize', 'Pclass_2', 'Pclass_3',
       'Sex_1', 'IsLonely_1'],
      dtype='object')

In [47]:
probs_test = results.predict(X_test)

probs_test

,0
0,0.106130
1,0.720225
2,0.208173
3,0.125418
4,0.406797
...,...
413,0.043921
414,0.841788
415,0.088486
416,0.111393


In [48]:
labels_test = np.array(['no ']*len(probs_test))
labels_test[probs_test > 0.5] = 'yes'

dead_live_test = y_test.replace({0:'no ', 1:'yes'})

confusion_table(labels_test, dead_live_test)

Truth,no,yes
Predicted,,
no,244,15
yes,22,137


In [49]:
print(accuracy_score(labels_test, dead_live_test))

0.9114832535885168


In [50]:

# Supondo que você tenha o PassengerId correspondente
submission = pd.DataFrame({
    'PassengerId': test['PassengerId'],  # Ids dos passageiros do conjunto de teste
    'Survived': (labels_test == 'yes').astype(int)  # 1 para 'yes', 0 para 'no'
})

# Salvar o arquivo submission.csv
submission.to_csv('submission.csv', index=False)


In [56]:
!kaggle competitions submit -c titanic -f submission.csv -m "Minha primeira submissão"

100% 2.77k/2.77k [00:00<00:00, 14.8kB/s]
Successfully submitted to Titanic - Machine Learning from Disaster

In [52]:
pd.read_csv('/content/submission.csv')

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


##sklearn

In [57]:
scaler = StandardScaler()
X_test = scaler.fit_transform(X_test)

y_predict = logistic_model.predict(X_test)


In [58]:
print(confusion_matrix(y_test,y_predict))

[[245  21]
 [ 20 132]]


In [59]:
print(accuracy_score(y_test,y_predict))

0.9019138755980861
